### Install the necessary packages:

pip install --upgrade gspread zenserp oauth2client

### Create an authentication for Google Drive: 
https://gspread.readthedocs.io/en/latest/oauth2.html

### Activate Google Drive API and Google Sheets API for your project
https://console.developers.google.com/project

### Required Files:
- token.json (Google Drive Authentication)
- apikey (Zenserp API Authentication ([https://app.zenserp.com/])

With this project we will create a spreadsheet and update the results of our rankwatch activities in a weekly spreadsheet.

In [67]:
# Initialize the required authentications

import zenserp
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('token.json', scope)
gc = gspread.authorize(credentials)

In [68]:
SPREADSHEET_NAME = "Zenserp Rank Watch" # The name of the spreadsheet
KEYWORDS = {
    "Pied Piper": ["https://silicon-valley.fandom.com"],
    "HBO Silicon Valley": ["https://www.imdb.com", "https://www.indiewire.com"],
} # A list of "keyword": [domain1, domain2, ...] pairs to check
ZENSERP_API_KEY = open('apikey', 'r').read().strip() # the apikey for zenserp.com

In [69]:
try:
    sh = gc.open(SPREADSHEET_NAME)
except gspread.SpreadsheetNotFound:
    sh = gc.create(SPREADSHEET_NAME)

from datetime import datetime
calendar_week = 'Week ' + str(datetime.now().isocalendar()[1])
if calendar_week not in [sheet.title for sheet in sh.worksheets()]:
    worksheet = sh.add_worksheet(title=calendar_week, rows="100", cols="10")
else:
    worksheet = sh.worksheet(calendar_week)
    
worksheet.update_acell('A1', 'Keyword')
worksheet.update_acell('B1', 'Domain')
worksheet.update_acell('C1', 'Rank')

api_client = zenserp.Client(ZENSERP_API_KEY)
i = 2
for keyword in KEYWORDS.keys():
    params = (('q', keyword), ('search_engine', 'google.com'))
    results = api_client.search(params)
    
    for domain in KEYWORDS[keyword]:
        for serp_result in results['results']['left']:
            if serp_result['type'] == "organic":
                if domain in serp_result['url']:
                
                    worksheet.update_cell(i, 1, keyword)
                    worksheet.update_cell(i, 2, domain)
                    worksheet.update_cell(i, 3, serp_result['position'])
                    
                    i += 1

In [70]:
# share it with yourself, if you are using a service account
sh.share('office@zenserp.com', perm_type='user', role='writer')